In [1]:
from bs4 import BeautifulSoup
import re
import requests

import pandas as pd 

import spacy
import spacy.cli

In [2]:
from time import sleep
from random import randint

In [3]:
def cria_dicionario():
    global lista_sent
    #print(lista_sent[0])
    s, o, h = lista_sent.pop(0)
    #print(s)
    #verifica se a palavra está no dicionario
    if s in dic_sentimentos:
        return
    
    if h > d:
        dic_sentimentos[s] =  o
        return
    
    #caso as verificações estejam ok, adiciona no dicionário e faz a busca 
    
    dic_sentimentos[s] =  o
    
    url = "https://www.sinonimos.com.br/" + s
    soup = BeautifulSoup(requests.get(url).text, "html5lib")
    retorno = soup.find_all('a', class_ ='sinonimo')
    lista_sent = lista_sent + list(pd.Series(retorno).apply(lambda x: (str(x).split('>')[1][:-3], o, h+1)))
    
    #sleep(randint(1,3))
    
    cria_dicionario() 

In [4]:
dic_sentimentos = {}
lista_sent = [('alegria','alegria', 0), ('tristeza','tristeza', 0), ('raiva','raiva', 0), ('nojo','nojo', 0), ('medo','medo', 0)]
#lista_sent = [('raiva','raiva', 0), ('nojo','nojo', 0), ('medo','medo', 0)]
d = 1

while lista_sent != []:
    cria_dicionario()

/home/karine/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [5]:
(pd.DataFrame.from_dict(dic_sentimentos, orient='index')).to_csv('dic_sentimento_3_niveis.txt', sep='\t' )

In [6]:
pd.DataFrame.from_dict(dic_sentimentos, orient='index', columns = ['sent'])['sent'].value_counts()

alegria     328
tristeza    318
raiva       244
nojo        151
medo        111
Name: sent, dtype: int64